# Install Required Packages

In [1]:
# !git clone https://github.com/AbdelrhmanElnenaey/ASR_for_egyptian_dialect
%cd ASR_for_egyptian_dialect

/kaggle/working/ASR_for_egyptian_dialect


In [ ]:
!pip install wget
!apt-get install -y sox libsndfile1 ffmpeg
!pip install text-unidecode

BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

!pip install torchaudio -f https://download.pytorch.org/whl/torch_stable.html
!pip install gdown
!pip install demucs

# Modify Extract Embeddings Function in NeMo Clustering Diarizer

In [6]:
import site
import os
site_packages_path = site.getsitepackages()[0]
file_path = os.path.join(site_packages_path, 'nemo', 'collections', 'asr', 'models', 'clustering_diarizer.py')
print(f"File path: {file_path}")


File path: /opt/conda/lib/python3.10/site-packages/nemo/collections/asr/models/clustering_diarizer.py


In [7]:
modifications = """
def _extract_embeddings(self, manifest_file: str, scale_idx: int, num_scales: int):
    print("INSIDE MODIFIED EXTRACT EMBEDDINGS..")
    import nemo.collections.asr as nemo_asr
    self._ecapa_tdnn_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained(model_name='ecapa_tdnn')
    self._ecapa_tdnn_model.eval()

    logging.info("Extracting embeddings for Diarization")
    self._setup_spkr_test_data(manifest_file)
    self.embeddings = {}
    self._speaker_model.eval()
    self.time_stamps = {}

    all_embs = torch.empty([0])
    for test_batch in tqdm(
        self._speaker_model.test_dataloader(),
        desc=f'[{scale_idx+1}/{num_scales}] extract embeddings',
        leave=True,
        disable=not self.verbose,
    ):
        test_batch = [x.to(self._speaker_model.device) for x in test_batch]
        audio_signal, audio_signal_len, labels, slices = test_batch
        with autocast():
            _, embs = self._speaker_model.forward(input_signal=audio_signal, input_signal_length=audio_signal_len)
            embs_ecapa = self._ecapa_tdnn_model.forward(input_signal=audio_signal, input_signal_length=audio_signal_len)
            embs = torch.cat((embs, embs_ecapa[0]), dim=-1)
            emb_shape = embs.shape[-1]
            embs = embs.view(-1, emb_shape)
            all_embs = torch.cat((all_embs, embs.cpu().detach()), dim=0)
        del test_batch

    with open(manifest_file, 'r', encoding='utf-8') as manifest:
        for i, line in enumerate(manifest.readlines()):
            line = line.strip()
            dic = json.loads(line)
            uniq_name = get_uniqname_from_filepath(dic['audio_filepath'])
            if uniq_name in self.embeddings:
                self.embeddings[uniq_name] = torch.cat((self.embeddings[uniq_name], all_embs[i].view(1, -1)))
            else:
                self.embeddings[uniq_name] = all_embs[i].view(1, -1)
            if uniq_name not in self.time_stamps:
                self.time_stamps[uniq_name] = []
            start = dic['offset']
            end = start + dic['duration']
            self.time_stamps[uniq_name].append([start, end])

    if self._speaker_params.save_embeddings:
        embedding_dir = os.path.join(self._speaker_dir, 'embeddings')
        if not os.path.exists(embedding_dir):
            os.makedirs(embedding_dir, exist_ok=True)

        prefix = get_uniqname_from_filepath(manifest_file)
        name = os.path.join(embedding_dir, prefix)
        self._embeddings_file = name + f'_embeddings.pkl'
        pkl.dump(self.embeddings, open(self._embeddings_file, 'wb'))
        logging.info("Saved embedding files to {}".format(embedding_dir))

"""

if os.path.isfile(file_path):
    with open(file_path, 'r') as file:
        file_data = file.read()

    # find the location to insert the new function
    start_idx = file_data.find("def _extract_embeddings")
    end_idx = file_data.find("def ", start_idx + 1) if file_data.find("def ", start_idx + 1) != -1 else len(file_data)

    indent = " " * 4
    indented_modifications = "\n".join([indent + line if line.strip() else line for line in modifications.split('\n')])

    # create new file content
    new_file_data = file_data[:start_idx] + indented_modifications + "\n\n" + indent + file_data[end_idx:]

    # Write the modified content back to the file
    with open(file_path, 'w') as file:
        file.write(new_file_data)

    print(f"Modified function in {file_path}.")
else:
    print(f"File not found: {file_path}")

Modified function in /opt/conda/lib/python3.10/site-packages/nemo/collections/asr/models/clustering_diarizer.py.


In [6]:
## ONLY ON KAGGLE
## You will need to restart the kernel after running this cell
## Do not make factory reset, just restart the kernel


# package_path = os.path.dirname(package_name.__file__)
file_to_edit = '/opt/conda/lib/python3.10/site-packages/nemo/collections/asr/parts/submodules/rnnt_decoding.py'  # Update 'subdir' and 'file.py' as needed

# Read the file
with open(file_to_edit, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Make your edits (for example, replacing a specific line)
for i, line in enumerate(lines):
#     print(line)
    if '# Copyright (c) 2020, NVIDIA CORPORATION.  All rights reserved.' in line:
        lines[i] = 'import copy\n'
    
# Write the file back
with open(file_to_edit, 'w', encoding='utf-8') as file:
    file.writelines(lines)


# You Will Need To Restart the Kernel After the Modification

In [ ]:
os._exit(00)


# Imports

In [1]:
import gdown
import argparse
import torch
import soundfile as sf
import os
import gdown
import nemo.collections.asr as nemo_asr
from ruamel.yaml import YAML
from omegaconf import OmegaConf, open_dict
import json
import subprocess
import logging
from pydub import AudioSegment
import numpy as np
from IPython.display import Audio, display
import librosa
import wget
import matplotlib.pyplot as plt
import nemo
import glob
import pprint
import shutil

In [4]:
%cd /kaggle/working/ASR_for_egyptian_dialect

/kaggle/working/ASR_for_egyptian_dialect


# ASR Inference

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_asr_model(ckpt_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    if(not os.path.exists(ckpt_path)):
        url = "https://drive.google.com/file/d/1faLSvzXVcZd_lvBXxxdWYyBGyGnC2ijL/view?usp=drive_link"
        gdown.download(url, ckpt_path, quiet=False, fuzzy=True)

    config_path = '/kaggle/input/config2/FC-transducer-inference.yaml'
    yaml = YAML(typ='safe')
    with open(config_path) as f:
        params = yaml.load(f)
    params['model'].pop('test_ds')
    conf = OmegaConf.create(params)

    model = nemo_asr.models.EncDecRNNTBPEModel(cfg=conf['model']).to(device)
    decoding_cfg = model.cfg.decoding
    with open_dict(decoding_cfg):
        decoding_cfg.preserve_alignments = True
        decoding_cfg.compute_timestamps = True
        decoding_cfg.strategy = 'greedy'
#         decoding_cfg.beam.beam_size = 3 
        model.change_decoding_strategy(decoding_cfg)
    model.load_state_dict(torch.load(ckpt_path)['state_dict'])
    model.eval()
    return model
    

def create_parser():
    parser = argparse.ArgumentParser(description="ASR Inference")
    parser.add_argument("--asr_model", type=str, help="Path to the ASR model checkpoint", default="asr_model.ckpt")
    parser.add_argument("--data_dir", type=str, help="Path to the directory containing test data", default="data/adapt")
    parser.add_argument("--output", type=str, help="Path to the output file", default="results.csv")
    return parser

def infere(model, audio):
    hypotheses = model.transcribe([audio], return_hypotheses=True)
    if type(hypotheses) == tuple and len(hypotheses) == 2:
        hypotheses = hypotheses[0]
    return hypotheses


# Generate Manifest File for Diarization

In [ ]:
%cd /kaggle/working/ASR_for_egyptian_dialect
data_dir = '/kaggle/input/diarization/wavs'
with open("audio_files.txt", "w") as f:
    for file in os.listdir(data_dir):
        f.write(os.path.join(os.getcwd(), data_dir, file) + "\n")
        

!python /kaggle/working/ASR_for_egyptian_dialect/NeMo/scripts/speaker_tasks/pathfiles_to_diarize_manifest.py \
--paths2audio_files audio_files.txt \
--manifest_filepath test_manifest.json \
--add_duration


# Preprocessing
The following script takes the original manifest and applies source separation to the wav files and generates a new manifest

In [ ]:
input_manifest_path = '/kaggle/working/ASR_for_egyptian_dialect/test_manifest.json'
output_manifest_path = '/kaggle/working/ASR_for_egyptian_dialect/test_manifest_vocals.json'
temp_output_dir = 'temp_outputs'
mono_output_dir = '/kaggle/working/ASR_for_egyptian_dialect/temp_outputs_mono'
os.makedirs(mono_output_dir, exist_ok=True)


In [ ]:
with open(input_manifest_path, 'r') as f:
    manifest = [json.loads(line) for line in f]

new_manifest = []
for entry in manifest:
    audio_filepath = entry['audio_filepath']
    base_name = os.path.splitext(os.path.basename(audio_filepath))[0]

    try:
        result = subprocess.run(
            [
                "python3",
                "-m",
                "demucs.separate",
                "-n",
                "htdemucs",
                "--two-stems=vocals",
                audio_filepath,
                "-o",
                temp_output_dir
            ],
            check=True,
            capture_output=True,
            text=True
        )
        # rename the output file to match the original base name (for compatibility with NeMo)
        original_vocal_filepath = os.path.join(temp_output_dir, "htdemucs", base_name, "vocals.wav")
        vocal_filepath = os.path.join(temp_output_dir, "htdemucs", f"{base_name}.wav")
        if os.path.exists(original_vocal_filepath):
            os.rename(original_vocal_filepath, vocal_filepath)
        else:
            raise FileNotFoundError(f"Expected output file not found: {original_vocal_filepath}")
    except (subprocess.CalledProcessError, FileNotFoundError) as e:
        logging.warning(f"Source splitting failed for {audio_filepath}, using original audio file. Error: {str(e)}")
        vocal_filepath = audio_filepath

    # Convert to mono
    sound = AudioSegment.from_file(vocal_filepath).set_channels(1)
    mono_vocal_filepath = os.path.join(mono_output_dir, f"{base_name}.wav")
    sound.export(mono_vocal_filepath, format="wav")

    # Update the manifest entry with the new audio filepath
    new_entry = entry.copy()
    new_entry['audio_filepath'] = mono_vocal_filepath
    new_manifest.append(new_entry)

# Save the new manifest
with open(output_manifest_path, 'w') as f:
    for entry in new_manifest:
        json.dump(entry, f)
        f.write('\n')

# Run ASR Inference
Returns

`word_hyp` : A list of all the words output from the ASR

`word_ts_hyp`: A list of tuples, each tuple contains the start and end timestamp of the corresponding word.

In [3]:
%cd /kaggle/working/ASR_for_egyptian_dialect
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = '/kaggle/input/diarization/wavs'
asr_model = load_asr_model('/kaggle/input/asr-model/asr_model.ckpt')
asr_model.to(device)


word_hyp = {}
word_ts_hyp = {}


with open('/kaggle/working/results2.csv', "w+", encoding='utf-8') as fp:
    fp.write("audio,transcript,start,end\n")

for filename in os.listdir(data_dir):
    if filename.endswith('.wav'):
        audio, sr = sf.read(os.path.join(data_dir, filename), dtype='float32')
        audio_path = os.path.join(data_dir, filename)
        with torch.no_grad():
            rv = infere(model=asr_model, audio=audio_path)

        timestamp_dict = rv[0].timestep
        time_stride = 8 * asr_model.cfg.preprocessor.window_stride
        word_timestamps = timestamp_dict['word']

        words = []
        word_timestamps_list = []

        with open('/kaggle/working/results2.csv', "a+") as fp:
            for i, stamp in enumerate(word_timestamps):
                start = stamp['start_offset'] * time_stride
                end = stamp['end_offset'] * time_stride
                transcription = stamp['word']

                words.append(transcription)
                word_timestamps_list.append([start, end])


                fp.write(f"{filename},{transcription},{start},{end}\n")
            unique_id = os.path.splitext(filename)[0] 
            word_hyp[unique_id] = words
            word_ts_hyp[unique_id] = word_timestamps_list

/kaggle/working/ASR_for_egyptian_dialect
cuda
[NeMo I 2024-07-24 16:36:02 mixins:173] Tokenizer SentencePieceTokenizer initialized with 256 tokens


[NeMo W 2024-07-24 16:36:03 audio_to_text_dataset:830] Could not load dataset as `manifest_filepath` was None. Provided config : {'manifest_filepath': None, 'sample_rate': 16000, 'batch_size': 32, 'shuffle': True, 'num_workers': 8, 'pin_memory': True, 'max_duration': 16.7, 'min_duration': 0.1, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'fully_randomized', 'bucketing_batch_size': None}
[NeMo W 2024-07-24 16:36:03 audio_to_text_dataset:830] Could not load dataset as `manifest_filepath` was None. Provided config : {'manifest_filepath': None, 'sample_rate': 16000, 'batch_size': 32, 'shuffle': False, 'use_start_end_token': False, 'num_workers': 8, 'pin_memory': True}


[NeMo I 2024-07-24 16:36:03 features:305] PADDING: 0


[NeMo W 2024-07-24 16:36:04 nemo_logging:349] /opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2024-07-24 16:36:04 rnnt_models:224] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2024-07-24 16:36:04 rnnt_models:224] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2024-07-24 16:36:04 rnnt_models:224] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2024-07-24 16:36:04 rnnt_models:224] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2024-07-24 16:36:04 rnnt_bpe_models:492] Changed decoding strategy to 
    model_type: rnnt
    strategy: greedy
    compute_hypothesis_token_set: false
    preserve_alignments: true
    confidence_cfg:
      preserve_frame_confidence: false
      preserve_token_confidence: false
      preserve_word_confidence: false
      exclude_blank: true
      aggregation: min
      tdt_include_durati

Transcribing: 100%|██████████| 1/1 [00:08<00:00,  8.40s/it]


# Diarization Inference

In [8]:
pp = pprint.PrettyPrinter(indent=4)

data_dir = "/kaggle/working/diarization_output"
os.makedirs('/kaggle/working/diarization_output',exist_ok=True)

DOMAIN_TYPE = "general" # Can be meeting or telephonic based on domain type of the audio file
CONFIG_FILE_NAME = f"diar_infer_{DOMAIN_TYPE}.yaml"

CONFIG_URL = f"https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/{CONFIG_FILE_NAME}"

if not os.path.exists(os.path.join(data_dir,CONFIG_FILE_NAME)):
    CONFIG = wget.download(CONFIG_URL, data_dir)
else:
    CONFIG = os.path.join(data_dir,CONFIG_FILE_NAME)

cfg = OmegaConf.load(CONFIG)
cfg.diarizer.collar = 0
cfg.diarizer.vad.parameters.window_length_in_sec=0.63 #1.2
cfg.diarizer.vad.parameters.shift_length_in_sec=0.05 #0.3
cfg.diarizer.manifest_filepath = '/kaggle/working/ASR_for_egyptian_dialect/test_manifest_vocals.json'

pretrained_speaker_model='titanet_large'
cfg.diarizer.out_dir = data_dir #Directory to store intermediate files and prediction outputs
cfg.diarizer.speaker_embeddings.model_path = pretrained_speaker_model
cfg.diarizer.clustering.parameters.oracle_num_speakers=False

cfg.diarizer.vad.model_path = 'vad_multilingual_marblenet'
cfg.diarizer.oracle_vad = False
cfg.diarizer.asr.parameters.asr_based_vad = False

print(OmegaConf.to_yaml(cfg))

name: ClusterDiarizer
num_workers: 1
sample_rate: 16000
batch_size: 64
device: null
verbose: true
diarizer:
  manifest_filepath: /kaggle/input/manifest/input_manifest.json
  out_dir: /kaggle/working/diarization_output3
  oracle_vad: false
  collar: 0
  ignore_overlap: true
  vad:
    model_path: vad_multilingual_marblenet
    external_vad_manifest: null
    parameters:
      window_length_in_sec: 0.95
      shift_length_in_sec: 0.05
      smoothing: false
      overlap: 0.5
      onset: 0.5
      offset: 0.3
      pad_onset: 0.2
      pad_offset: 0.2
      min_duration_on: 0.5
      min_duration_off: 0.5
      filter_speech_first: true
  speaker_embeddings:
    model_path: titanet_large
    parameters:
      window_length_in_sec:
      - 1.9
      - 1.2
      - 0.5
      shift_length_in_sec:
      - 0.95
      - 0.6
      - 0.25
      multiscale_weights:
      - 1
      - 1
      - 1
      save_embeddings: true
  clustering:
    parameters:
      oracle_num_speakers: false
      max_nu

In [10]:
from nemo.collections.asr.parts.utils.diarization_utils import OfflineDiarWithASR
asr_diar_offline = OfflineDiarWithASR(cfg.diarizer)
diar_hyp, diar_score = asr_diar_offline.run_diarization(cfg, word_ts_hyp)

[NeMo I 2024-07-24 16:39:00 clustering_diarizer:127] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2024-07-24 16:39:00 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/vad_multilingual_marblenet/versions/1.10.0/files/vad_multilingual_marblenet.nemo to /root/.cache/torch/NeMo/NeMo_2.0.0rc2/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2024-07-24 16:39:01 common:826] Instantiating model from pre-trained checkpoint


[NeMo W 2024-07-24 16:39:01 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

[NeMo I 2024-07-24 16:39:01 features:305] PADDING: 16
[NeMo I 2024-07-24 16:39:01 save_restore_connector:275] Model EncDecClassificationModel was successfully restored from /root/.cache/torch/NeMo/NeMo_2.0.0rc2/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2024-07-24 16:39:01 clustering_diarizer:160] Loading pretrained titanet_large model from NGC
[NeMo I 2024-07-24 16:39:01 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/titanet_large/versions/v1/files/titanet-l.nemo to /root/.cache/torch/NeMo/NeMo_2.0.0rc2/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2024-07-24 16:39:03 common:826] Instantiating model from pre-trained checkpoint


[NeMo W 2024-07-24 16:39:03 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2024-07-24 16:39:03 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2024-07-24 16:39:03 features:305] PADDING: 16
[NeMo I 2024-07-24 16:39:04 save_restore_connector:275] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_2.0.0rc2/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.


...

[NeMo I 2024-07-24 16:40:02 clustering_diarizer:397] Saved embedding files to /kaggle/working/diarization_output3/speaker_outputs/embeddings


clustering: 100%|██████████| 1/1 [00:00<00:00,  1.00it/s]

[NeMo I 2024-07-24 16:40:03 clustering_diarizer:467] Outputs are saved in /kaggle/working/diarization_output3 directory



[NeMo W 2024-07-24 16:40:03 nemo_logging:349] /opt/conda/lib/python3.10/site-packages/pyannote/metrics/utils.py:200: UserWarning: 'uem' was approximated by the union of 'reference' and 'hypothesis' extents.
      warnings.warn(
    


[NeMo I 2024-07-24 16:40:03 der:176] Cumulative Results for collar 0 sec and ignore_overlap True: 
     FA: 0.0143	 MISS 0.0004	                 Diarization ER: 0.2303	, Confusion ER:0.2156


on audio sample:
der with titanet only: 23.6%
der with titanet and ecapa: 23.03%

on audio sample: der with titanet+ecapa and pca=192  0.95 window size: 22.97
same but 0.8 window size: 23.24%

In [11]:
trans_info_dict = asr_diar_offline.get_transcript_with_speaker_labels(diar_hyp, word_hyp, word_ts_hyp)

[NeMo I 2024-07-24 16:40:32 diarization_utils:876] Creating results for Session: audio_sample_20 n_spk: 3 
[NeMo I 2024-07-24 16:40:32 diarization_utils:749] Diarization with ASR output files are saved in: /kaggle/working/diarization_output3/pred_rttms


In [7]:
' '.join(word for word in word_hyp['audio_sample_20'])

'كل ما يتعلق بإسماعيل ياسين وبهجنا يمتعنا يستعيد ذكريات جميله و كمان خلينا نتطلع للأمام ليه لأن إحنا بنتعلم من هذه العموذ الفنيه ما بتموتش أبدا بنعيش على التراث الفني الجميل يعني فا بتحاول تشوفي تستفيدي من يعني هنا إنتي جدك على الشاشه يعني بيبهجنا ويمتعنا يعني بتحاول تشوفي تقولي لما ألقط حاجه من إسماعيل ياسين لما أستفيد حاجه من هذا التاريخ الناصع طبعا إحنا و إحنا صغيرين باباه كان على طول دي فرجنا على أفلامه أه حتى كان في فيلم مبيتعرضش كتير اللي هو إسماعيل ياسين في الجيش هو جايبه و كان حريص جدا إن إحنا نتفرج على أفلامه و كان بيكلمنا على هو في الشغل عامل إزاي يعني كان بيقولنا إن هو كان منضبط جدا بيحب المواعيد ملتزم إن هو شخصيه في الحقيقه جد و مثقف بيحب الايه الحاجات اللي كنت لما بتفرج على إسماعيل ياسين بقى باخدها أو ركز فيها كانت ساعات في إفيهات كده إفيه ذكي الإفيه الذكي أه يشوف إنتي كده بتنقوطي فعلا أه فكان يقول مثلا حاجات فعلا أنا الفيلم أنا مش قادره أفتكر إسمي حاجات مش مبشره فا في فيلم دلوقتي هو هي بتعكس قضيه و بتاع أساسي كان في فيلم ملون إسماعيلي ياسين أيوه العام الفيلم ده في كم إفيه

In [27]:
import os
import json

# Directories
input_dir = '/kaggle/working/diarization_output/pred_rttms'
output_dir = '/kaggle/working/diarization_VAD_ASR_results'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def process_json_files(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        if filename.endswith(".json") and "gecko" not in filename:
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            
            # Read the JSON file
            with open(input_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
            
            # Save the updated JSON to the new directory
            with open(output_path, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)

process_json_files(input_dir, output_dir)


# save in desired format

In [28]:
import os
import json

# Directories
input_dir = '/kaggle/working/diarization_VAD_ASR_results'
output_dir = '/kaggle/working/diarization_VAD_ASR_results_final'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def transform_json(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir,filename)
        with open(input_path, 'r', encoding='utf-8') as infile:
            data = json.load(infile)

        transformed_data = []

        for sentence in data.get('sentences', []):
            transformed_data.append({
                "start": float(sentence['start_time']),
                "end": float(sentence['end_time']),
                "speaker": int(sentence['speaker'].replace("speaker_", "")),
                "text": sentence['text']
            })
        output_path = os.path.join(output_dir,filename)
        with open(output_path, 'w', encoding='utf-8') as outfile:
            json.dump(transformed_data, outfile, ensure_ascii=False, indent=4)


transform_json(input_dir, output_dir)
